In [1]:
### ONLY RUN ON PRE-TRANSFORMED FILES!!!!!!!!!! ###

#########################
# Version of DNN using Mirco Config
# Set to take in Robust tranformed file
# Runs net and plots
############################

In [2]:
import numpy
import h5py
import time
import os, sys
import random
from collections import OrderedDict
import itertools
import matplotlib.pyplot as plt
import argparse

## Define Variables (USE TRANSFORMED TRAINING FILE ONLY)

In [3]:
### CHANGE VARIABLES HERE ###

## RUN TrasnformFiles_OutputHDF5 first !!!! ### 
input_file = '/home/jovyan/Level5p_IC86.2013_genie_nue.012640.300.transformed.hdf5' #'Level5_IC86.2013_genie_nue.012640.100.cascade.lt60_vertexDC.transformed.hdf5'
filename = 'nue_cascade_test_play' #Output model name
num_epochs = 5
dropout = 0.2
save = True
save_folder_name = filename + "/"
old_model_name = "old_model.hdf5" #If you want to read in an older model before starting training. NOTE: CURRENTLY COMMENTED OUT!

DC_drop_value = dropout
IC_drop_value = dropout
connected_drop_value = dropout

In [4]:
# IMPORT FILE #
f = h5py.File(input_file, 'r')
Y_train = f['Y_train'][:]
Y_test = f['Y_test'][:]
X_train_DC = f['X_train_DC'][:]
X_test_DC = f['X_test_DC'][:]
X_train_IC = f['X_train_IC'][:]
X_test_IC = f['X_test_IC'][:]
f.close()
del f

In [5]:
# Y train has 11 variables, want to train on energy only right now
# Labels: [ nu energy, nu zenith, nu azimuth, nu time, nu x, nu y, nu z, track length (0 for cascade), isTrack (track = 1, cascasde = 0), 
#         flavor, type (anti = 1), isCC (CC=1, NC = 0)]

Y_train = Y_train[:,0] # ENERGY ONLY
#Y_test = Y_test[:,0] # ENERGY ONLY

# Return features and labels, to be used for network
num_features_DC = X_train_DC.shape[-1]
num_features_IC = X_train_IC.shape[-1]
num_labels = 1 #Y_train.shape[-1] ## NEED TO CHANGE MANUALLY!
print("Training set: %i, Testing set: %i"%(len(Y_train),len(Y_test)))

Training set: 258677, Testing set: 28742


## Build Network

In [6]:
### BUILD THE NETWORK ###

from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Lambda
from keras.layers import concatenate
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras import initializers
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

# DEEP CORE #
print("Train Data DC", X_train_DC.shape)
strings = X_train_DC.shape[1]
dom_per_string = X_train_DC.shape[2]
dom_variables = X_train_DC.shape[3]

# Conv DC + batch normalization, later dropout and maxpooling
input_DC = Input(shape=(strings, dom_per_string, dom_variables))

conv1_DC = Conv2D(100,kernel_size=(strings,5),padding='same',activation='tanh')(input_DC) #tanh
batch1_DC = BatchNormalization()(conv1_DC)
pool1_DC = MaxPooling2D(pool_size=(1,2))(batch1_DC)
drop1_DC = Dropout(DC_drop_value)(pool1_DC)

conv2_DC = Conv2D(100,kernel_size=(strings,7),padding='same',activation='relu')(drop1_DC) #relu
batch2_DC = BatchNormalization()(conv2_DC)
drop2_DC = Dropout(DC_drop_value)(batch2_DC)

conv3_DC = Conv2D(100,kernel_size=(strings,7),padding='same',activation='relu')(drop2_DC)
batch3_DC = BatchNormalization()(conv3_DC)
drop3_DC = Dropout(DC_drop_value)(batch3_DC)

conv4_DC = Conv2D(100,kernel_size=(strings,3),padding='valid',activation='relu')(drop3_DC)
batch4_DC = BatchNormalization()(conv4_DC)
pool4_DC = MaxPooling2D(pool_size=(1,2))(batch4_DC)
drop4_DC = Dropout(DC_drop_value)(pool4_DC)

conv5_DC = Conv2D(100,kernel_size=(1,7),padding='same',activation='relu')(drop4_DC)
batch5_DC = BatchNormalization()(conv5_DC)
drop5_DC = Dropout(DC_drop_value)(batch5_DC)

conv6_DC = Conv2D(100,kernel_size=(1,7),padding='same',activation='relu')(drop5_DC)
batch6_DC = BatchNormalization()(conv6_DC)
drop6_DC = Dropout(DC_drop_value)(batch6_DC)

conv7_DC = Conv2D(100,kernel_size=(1,1),padding='same',activation='relu')(drop6_DC)
batch7_DC = BatchNormalization()(conv7_DC)
drop7_DC = Dropout(DC_drop_value)(batch7_DC)

conv8_DC = Conv2D(100,kernel_size=(1,1),padding='same',activation='relu')(drop7_DC)
batch8_DC = BatchNormalization()(conv8_DC)
drop8_DC = Dropout(DC_drop_value)(batch8_DC)

flat_DC = Flatten()(drop8_DC)

# ICECUBE NEAR DEEPCORE #
print("Train Data IC", X_train_IC.shape)
strings_IC = X_train_IC.shape[1]
dom_per_string_IC = X_train_IC.shape[2]
dom_variables_IC = X_train_IC.shape[3]

# Conv DC + batch normalization, later dropout and maxpooling
input_IC = Input(shape=(strings_IC, dom_per_string_IC, dom_variables_IC))

conv1_IC = Conv2D(100,kernel_size=(strings_IC,5),padding='same',activation='tanh')(input_IC)
batch1_IC = BatchNormalization()(conv1_IC)
pool1_IC = MaxPooling2D(pool_size=(1,2))(batch1_IC)
drop1_IC = Dropout(IC_drop_value)(pool1_IC)

conv2_IC = Conv2D(100,kernel_size=(strings_IC,7),padding='same',activation='relu')(drop1_IC)
batch2_IC = BatchNormalization()(conv2_IC)
drop2_IC = Dropout(IC_drop_value)(batch2_IC)

conv3_IC = Conv2D(100,kernel_size=(strings_IC,7),padding='same',activation='relu')(drop2_IC)
batch3_IC = BatchNormalization()(conv3_IC)
drop3_IC = Dropout(IC_drop_value)(batch3_IC)

conv4_IC = Conv2D(100,kernel_size=(strings_IC,3),padding='valid',activation='relu')(drop3_IC)
batch4_IC = BatchNormalization()(conv4_IC)
pool4_IC = MaxPooling2D(pool_size=(1,2))(batch4_IC)
drop4_IC = Dropout(IC_drop_value)(pool4_IC)

conv5_IC = Conv2D(100,kernel_size=(1,7),padding='same',activation='relu')(drop4_IC)
batch5_IC = BatchNormalization()(conv5_IC)
drop5_IC = Dropout(IC_drop_value)(batch5_IC)

conv6_IC = Conv2D(100,kernel_size=(1,7),padding='same',activation='relu')(drop5_IC)
batch6_IC = BatchNormalization()(conv6_IC)
drop6_IC = Dropout(IC_drop_value)(batch6_IC)

conv7_IC = Conv2D(100,kernel_size=(1,1),padding='same',activation='relu')(drop6_IC)
batch7_IC = BatchNormalization()(conv7_IC)
drop7_IC = Dropout(IC_drop_value)(batch7_IC)

conv8_IC = Conv2D(100,kernel_size=(1,1),padding='same',activation='relu')(drop7_IC)
batch8_IC = BatchNormalization()(conv8_IC)
drop8_IC = Dropout(IC_drop_value)(batch8_IC)

flat_IC = Flatten()(drop8_IC)

# PUT TOGETHER #
concatted = concatenate([flat_DC, flat_IC])

full1 = Dense(300,activation='relu')(concatted)
batch1_full = BatchNormalization()(full1)
dropf = Dropout(connected_drop_value)(batch1_full)

output = Dense(num_labels,activation='linear')(dropf)
#batch2_full = BatchNormalization()(output)
#lambda_layer = Lambda(lambda x: (x*20.)/1.+3)(output)
lambda_layer = Lambda(lambda x: (x*335)/1.)(output)
model_DC = Model(inputs=[input_DC,input_IC],outputs=lambda_layer)

#print(model_DC.summary())

Using TensorFlow backend.


Train Data DC (258677, 8, 60, 5)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train Data IC (258677, 7, 60, 5)


In [10]:
## Compile ##
model_DC.compile(loss='mean_squared_error',
              optimizer=Adam(lr=1.e-3), #0.00001
              metrics=['mean_squared_error'])

In [11]:
#If you want to load a model (already trained) in to add to
#model_DC.load_weights(old_model_name)

In [12]:
## Run neural network and record time ##
t0 = time.time()
network_history = model_DC.fit([X_train_DC, X_train_IC], Y_train,
                            batch_size=256,
                            validation_split=0.25,
                            epochs=num_epochs,
                            callbacks = [EarlyStopping(patience=6), ModelCheckpoint('current_model_while_running.hdf5')],
                            verbose=1)

t1 = time.time()
print("This took me %f minutes"%((t1-t0)/60.))

Train on 194007 samples, validate on 64670 samples
Epoch 1/5
194007/194007 [==============================] - 271s 1ms/step - loss: 82.9696 - mean_squared_error: 82.9696 - val_loss: 68.2617 - val_mean_squared_error: 68.2617
Epoch 2/5
194007/194007 [==============================] - 258s 1ms/step - loss: 73.4027 - mean_squared_error: 73.4027 - val_loss: 69.4311 - val_mean_squared_error: 69.4311
Epoch 3/5
194007/194007 [==============================] - 260s 1ms/step - loss: 67.7466 - mean_squared_error: 67.7466 - val_loss: 64.8973 - val_mean_squared_error: 64.8973
Epoch 4/5
194007/194007 [==============================] - 260s 1ms/step - loss: 64.1719 - mean_squared_error: 64.1719 - val_loss: 63.9818 - val_mean_squared_error: 63.9818
Epoch 5/5
194007/194007 [==============================] - 261s 1ms/step - loss: 60.4141 - mean_squared_error: 60.4141 - val_loss: 57.5427 - val_mean_squared_error: 57.5427
This took me 22.060043 minutes


In [14]:
score = model_DC.evaluate([X_test_DC,X_test_IC], Y_test[:,0], batch_size=256)
print("final score on test data: loss: {:.4f} / accuracy: {:.4f}".format(score[0], score[1]))
print(network_history.history.keys())
print(score)

28742/28742 [==============================] - 12s 409us/step
final score on test data: loss: 58.2880 / accuracy: 58.2880
dict_keys(['val_loss', 'val_mean_squared_error', 'loss', 'mean_squared_error'])
[58.28800062562203, 58.28800062562203]


In [ ]:
## SAVE FOR FUTURE USE! ##
model_DC.save("/mnt/scratch/micall12/training_files/%s_model.hdf5"%filename)

## Test Set - Predict Reconstructed Energy 

In [15]:
Y_test_predicted = model_DC.predict([X_test_DC,X_test_IC])

Y_test_all_labels = numpy.copy(Y_test)
Y_test_predicted_energy = numpy.reshape(Y_test_predicted, Y_test_predicted.shape[0])
Y_test_energy = numpy.reshape(Y_test[:,0], Y_test.shape[0]) #Only compare ENERGY, change Y_test[:,i] 

if save==True:
    if os.path.isdir(save_folder_name) != True:
        os.mkdir(save_folder_name)

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


## Define Plots

In [ ]:
def plot_history(network_history,save=False,savefolder=None):
    """
    Plot history of neural network's loss vs. epoch
    Recieves:
        network_history = array, saved metrics from neural network training
        save = optional, bool to save plot
        savefolder = optional, output folder to save to, if not in current dir
    Returns:
        one plot, saved to files
    """
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    #plt.yscale('log')
    plt.plot(network_history.history['loss'])
    plt.plot(network_history.history['val_loss'])
    plt.legend(['Training', 'Validation'])
    if save == True:
        plt.savefig("%sloss_vs_epochs.png"%savefolder)

In [ ]:
def plot_distributions_CCNC(truth_all_labels,truth,reco,save=False,savefolder=None):
    """
    Plot testing set distribution, with CC and NC distinguished
    Recieves:
        truth_all_labels = array, Y_test truth labels that have ALL values in them (need CC vs NC info)
        truth = array, Y_test truth labels
        reco = array, neural network prediction output
        save = optional, bool to save plot
        savefolder = optional, output folder to save to, if not in current dir
    Returns:
        1D histogram of energy distribution with sepearated CC and NC distinction
    """
    CC_mask = truth_all_labels[:,11] ==1
    NC_mask = truth_all_labels[:,11] ==0
    num_CC = sum(CC_mask)
    num_NC = sum(NC_mask)
    print("CC events: %i, NC events: %i, Percent NC: %.2f"%(num_CC,num_NC,float(num_NC/(num_CC+num_NC))*100.))

    plt.figure(figsize=(10,7))
    plt.title("True Energy Distribution")
    plt.hist(truth[CC_mask], bins=100,color='b',alpha=0.5,label="CC");
    plt.hist(truth[NC_mask], bins=100,color='g',alpha=0.5,label="NC");
    plt.xlabel("Energy (GeV)")
    plt.legend()
    if save:
        plt.savefig("%sTrueEnergyDistribution_CCNC.png"%savefolder)

    plt.figure(figsize=(10,7))
    plt.title("NN Energy Distribution")
    plt.hist(reco[CC_mask], bins=100,color='b', alpha=0.5, label="CC");
    plt.hist(reco[NC_mask], bins=100,color='g', alpha=0.5, label="NC");
    plt.xlabel("Energy (GeV)")
    plt.legend()
    if save:
        plt.savefig("%sNNEnergyDistribution_CCNC.png"%savefolder)

In [ ]:
def plot_resolution_CCNC(truth_all_labels,truth,reco,save=False,savefolder=None):
    """
    Plot testing set resolution of reconstruction - truth, with CC and NC distinguished
    Recieves:
        truth_all_labels = array, Y_test truth labels that have ALL values in them (need CC vs NC info)
        truth = array, Y_test truth labels
        reco = array, neural network prediction output
        save = optional, bool to save plot
        savefolder = optional, output folder to save to, if not in current dir
    Returns:
        1D histogram of reco - true with sepearated CC and NC distinction
    """
    CC_mask = truth_all_labels[:,11] ==1
    NC_mask = truth_all_labels[:,11] ==0
    num_CC = sum(CC_mask)
    num_NC = sum(NC_mask)
    print("CC events: %i, NC events: %i, Percent NC: %.2f"%(num_CC,num_NC,float(num_NC/(num_CC+num_NC))*100.))

    resolution = reco - truth
    resolution_fraction = (reco - truth)/truth
    resolution = numpy.array(resolution)
    resolution_fraction  = numpy.array(resolution_fraction)

    plt.figure(figsize=(10,7))
    plt.title("Energy Resolution")
    plt.hist(resolution[CC_mask], bins=50,color='b', alpha=0.5, label="CC");
    plt.hist(resolution[NC_mask], bins=50,color='g', alpha=0.5, label="NC");
    plt.xlabel("NN reconstruction - truth (GeV)")
    plt.legend()
    if save:
        plt.savefig("%sEnergyResolution_CCNC.png"%savefolder)

    plt.figure(figsize=(10,7))
    plt.title("Fractional Energy Resolution")
    plt.hist(resolution_fraction[CC_mask], bins=50,color='b', alpha=0.5, label="CC");
    plt.hist(resolution_fraction[NC_mask], bins=50,color='g', alpha=0.5, label="NC");
    plt.xlabel("(NN reconstruction - truth) / truth")
    plt.legend()
    if save:
        plt.savefig("%sEnergyResolutionFrac_CCNC.png"%savefolder)

In [ ]:
def plot_2D_prediction(truth, nn_reco, \
                        save=False,savefolder=None,syst_set="",\
                        use_fraction=False,bins=60,\
                        minenergy=0.,maxenergy=60.):
    """
    Plot testing set reconstruction vs truth
    Recieves:
        truth = array, Y_test truth
        nn_reco = array, neural network prediction output
        save = optional, bool to save plot
        savefolder = optional, output folder to save to, if not in current dir
    Returns:
        2D plot of True vs Reco
    """
    if not use_fraction:
        plt.figure(figsize=(10,7))
        cts,xbin,ybin,img = plt.hist2d(truth, nn_reco, bins=bins)
        plt.plot([minenergy,maxenergy],[minenergy,maxenergy],'k:')
        plt.xlim(minenergy,maxenergy)
        plt.ylim(minenergy,maxenergy)
        plt.xlabel("True Neutrino Energy (GeV)")
        plt.ylabel("NN Reconstruction Energy (GeV)")
        cbar = plt.colorbar()
        cbar.ax.set_ylabel('counts', rotation=90)
        plt.set_cmap('viridis_r')
        plt.title("Reconstruction (from NN) vs Truth for Energy")
        if save == True:
            plt.savefig("%sTruthReco_2DHist%s.png"%(savefolder,syst_set))

    if use_fraction:
        fractional_error = abs(truth - nn_reco)/ truth
        plt.figure(figsize=(10,7))
        plt.title("Fractional Error vs. Energy")
        plt.hist2d(truth, fractional_error,bins=60);
        plt.xlabel("True Energy (GeV)")
        plt.ylabel("Fractional Error")
        #plt.ylim(0,0.5)
        cbar = plt.colorbar()
        cbar.ax.set_ylabel('counts', rotation=90)
        if save == True:
            plt.savefig("%sTruthRecoFrac_2DHist%s.png"%(savefolder,syst_set))

In [ ]:
plot_history(network_history, save=save, savefolder=save_folder_name)
plot_2D_prediction(Y_test_energy, Y_test_predicted_energy, save=save, savefolder=save_folder_name)
plot_distributions_CCNC(Y_test_all_labels,Y_test_energy, Y_test_predicted_energy, save=save,s avefolder=save_folder_name)
plot_resolution_CCNC(Y_test_all_labels, Y_test_energy, Y_test_predicted_energy, save=save, savefolder=save_folder_name)